In [1]:
import tweepy
import csv
import time
import html.parser as hp
import pandas as pd
import nltk
import pickle
import numpy as np
import gensim
import re  # regular expressions (for playing with the text)
from gensim.parsing.preprocessing import STOPWORDS # common english "stop words" -- a, the, etc.
from gensim.parsing import PorterStemmer
from gensim import corpora, models
import nltk.data
import nltk

from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline
import spacy

from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models
import gensim
import os
from os import path
from time import sleep
import matplotlib.pyplot as plt
import random
import wordcloud
from wordcloud import WordCloud , STOPWORDS

import re
import pickle
import gensim
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import string


In [2]:
final=pd.read_excel('Strategy2010 -2018.xlsx')

In [3]:
final.shape

(131270, 4)

In [4]:
final.head()

,Unnamed: 0,filing_date,fund_CIK,principal_strategies
0,0001318148-10-001804,20101124,879569,?the fund pursues its objective by investing i...
1,0000356682-10-000007,20100201,356682,"the fund is a ""fund of funds"" that seeks to ..."
2,0001193125-10-282324,20101216,1174610,the fund invests in derivatives that proshar...
3,0001398344-10-000552,20100430,890453,...
4,0000950123-10-065147,20100713,1424958,"the fund, under normal circumstances..."


In [5]:
mylist=['lose','losing','lost','loses','loss']
value_list=[]
for row in final['principal_strategies']:
    before_one_word=[]
    before_two_word=[]
    before_words=[]
    row=str(row)
    row=row.translate(str.maketrans('', '', string.punctuation))
    words = row.lower().split()
    t = [i for i, x in enumerate(words) if x == "money"]
    for d in t:
        before_one_word.append(words[d-1])
        before_two_word.append(words[d-2])
    before_words=before_one_word+before_two_word
    if len([i for i in mylist if i in before_words])>0:
        value_list.append(1)
    else:
        value_list.append(0)
        #if ('lose' == (words[d-1]) or 'lose' == words[d-2]) or ('losing' == (words[d-1]) or 'losing' == (words[d-2])) or ('lost' == (words[d-1]) or 'lost' == (words[d-2])) or ('loses' == (words[d-1]) or 'loses' == (words[d-2])):
#             val=1
            #value_list.append(1)
#         else:
#             val=0
# #             #value_list.append(0)
#     value_list.append(val)

In [6]:
final['comp_money']=''

In [7]:
final['comp_money']=value_list

In [8]:
final.head()

,Unnamed: 0,filing_date,fund_CIK,principal_strategies,comp_money
0,0001318148-10-001804,20101124,879569,?the fund pursues its objective by investing i...,0
1,0000356682-10-000007,20100201,356682,"the fund is a ""fund of funds"" that seeks to ...",0
2,0001193125-10-282324,20101216,1174610,the fund invests in derivatives that proshar...,1
3,0001398344-10-000552,20100430,890453,...,0
4,0000950123-10-065147,20100713,1424958,"the fund, under normal circumstances...",0


In [9]:
final.comp_money.value_counts()

0    112277
1     18993
Name: comp_money, dtype: int64

In [10]:
mylist=['lose','losing','lost','loses','loss']
value_list=[]
for row in final['principal_strategies']:
    before_one_word=[]
    before_two_word=[]
    before_three_word=[]
    before_four_word=[]
    before_five_word=[]
    before_words=[]
    row=str(row)
    row=row.translate(str.maketrans('', '', string.punctuation))
    words = row.lower().split()
    t = [i for i, x in enumerate(words) if (x == "investment" or x=="investing" or x == "investments")]
    for d in t:
        before_one_word.append(words[d-1])
        if d >=1:
            before_two_word.append(words[d-2])
            before_three_word.append(words[d-3])
            before_four_word.append(words[d-4])
            before_five_word.append(words[d-5])
    before_words=before_one_word+before_two_word+before_three_word+before_four_word+before_five_word
    if len([i for i in mylist if i in before_words])>0:
        value_list.append(1)
    else:
        value_list.append(0)

In [11]:
final['comp_invest']=''

In [12]:
final['comp_invest']=value_list

In [13]:
final.head()

,Unnamed: 0,filing_date,fund_CIK,principal_strategies,comp_money,comp_invest
0,0001318148-10-001804,20101124,879569,?the fund pursues its objective by investing i...,0,0
1,0000356682-10-000007,20100201,356682,"the fund is a ""fund of funds"" that seeks to ...",0,0
2,0001193125-10-282324,20101216,1174610,the fund invests in derivatives that proshar...,1,1
3,0001398344-10-000552,20100430,890453,...,0,0
4,0000950123-10-065147,20100713,1424958,"the fund, under normal circumstances...",0,0


In [14]:
final.shape

(131270, 6)

In [15]:
final.comp_invest.value_counts()

0    118758
1     12512
Name: comp_invest, dtype: int64

In [16]:
der=['derivative','derivatives']

count_der=0
cnt_der=[]
for row in final['principal_strategies']:
    row=str(row)
    row=row.translate(str.maketrans('', '', string.punctuation))
    words = row.lower().split()
    #count_neg.append(len([i for i in tokens1 if i in list_Negative]))
    count_der = len([i for i in words if i in der])
    cnt_der.append(count_der)

In [17]:
final['Cnt_Derivative']=''
final['Cnt_Derivative']=cnt_der

In [18]:
final.head(10)

,Unnamed: 0,filing_date,fund_CIK,principal_strategies,comp_money,comp_invest,Cnt_Derivative
0,0001318148-10-001804,20101124,879569,?the fund pursues its objective by investing i...,0,0,1
1,0000356682-10-000007,20100201,356682,"the fund is a ""fund of funds"" that seeks to ...",0,0,0
2,0001193125-10-282324,20101216,1174610,the fund invests in derivatives that proshar...,1,1,10
3,0001398344-10-000552,20100430,890453,...,0,0,0
4,0000950123-10-065147,20100713,1424958,"the fund, under normal circumstances...",0,0,0
5,0001144204-10-055954,20101028,352667,"the fund invests primarily in high-quality, ...",0,0,0
6,0001104659-10-023089,20100429,837276,"under normal market conditions, the portfolio...",0,0,0
7,0000067590-10-000927,20101223,741350,. we look for investments that we think will i...,0,0,0
8,0001193125-10-240071,20101029,1450011,the fund seeks to achieve its investm...,0,0,2
9,0000950123-10-098784,20101101,809593,"ordinarily, at least 80% of the fu...",0,0,0


In [19]:
final.Cnt_Derivative.describe()

count    131270.000000
mean          1.326160
std           2.611942
min           0.000000
25%           0.000000
50%           0.000000
75%           2.000000
max          44.000000
Name: Cnt_Derivative, dtype: float64

In [20]:
#final.to_csv('Strategy_Compliance_Derivative2010-2018.csv', index=False)

In [21]:
final.shape

(131270, 7)

In [22]:
final.comp_money.value_counts()

0    112277
1     18993
Name: comp_money, dtype: int64

In [23]:
df=pd.read_csv('Strategy_Compliance_Derivative2010-2018_new.csv')

In [24]:
df.head()

,Unnamed: 0,filing_date,fund_CIK,principal_strategies,comp_money,comp_invest,Cnt_Derivative
0,0001318148-10-001804,20101124,879569,?the fund pursues its objective by investing i...,0,0,1
1,0000356682-10-000007,20100201,356682,"the fund is a ""fund of funds"" that seeks to ...",0,0,0
2,0001193125-10-282324,20101216,1174610,the fund invests in derivatives that proshar...,1,1,10
3,0001398344-10-000552,20100430,890453,...,0,0,0
4,0000950123-10-065147,20100713,1424958,"the fund, under normal circumstances...",0,0,0


In [25]:
df.shape

(131211, 7)

In [26]:
df.comp_money.value_counts()

0    112236
1     18975
Name: comp_money, dtype: int64

In [27]:
df.comp_invest.value_counts()

0    118709
1     12502
Name: comp_invest, dtype: int64

In [28]:
df.Cnt_Derivative.value_counts().head()

0    78665
1    18657
2    11498
3     5696
4     4103
Name: Cnt_Derivative, dtype: int64

In [30]:
df.Cnt_Derivative.describe()

count    131211.000000
mean          1.315804
std           2.539269
min           0.000000
25%           0.000000
50%           0.000000
75%           2.000000
max          31.000000
Name: Cnt_Derivative, dtype: float64

In [31]:
final.head()

,Unnamed: 0,filing_date,fund_CIK,principal_strategies,comp_money,comp_invest,Cnt_Derivative
0,0001318148-10-001804,20101124,879569,?the fund pursues its objective by investing i...,0,0,1
1,0000356682-10-000007,20100201,356682,"the fund is a ""fund of funds"" that seeks to ...",0,0,0
2,0001193125-10-282324,20101216,1174610,the fund invests in derivatives that proshar...,1,1,10
3,0001398344-10-000552,20100430,890453,...,0,0,0
4,0000950123-10-065147,20100713,1424958,"the fund, under normal circumstances...",0,0,0


In [32]:
final['Compliance']=final['comp_money']+final['comp_invest']

In [33]:
final.Compliance.value_counts()

0    111857
2     12092
1      7321
Name: Compliance, dtype: int64

In [34]:
7321+12092

19413

In [35]:
final.shape

(131270, 8)

In [36]:
19413/131270

0.1478860364134989